In [2]:
# for working code
import os
import numpy as np
import pandas as pd
import mlxtend
from math import ceil

df = pd.read_csv("./data/winequality-red-features.csv")

In [3]:
float_series = df['fixed acidity']
categorical_series = df['acid_level']
boolean_series = df['quality_2']
boolean_series2 = df['high_alc']
int_series = df['quality']